# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [5]:
import sys
import numpy as np
import pandas as pd
import sklearn

# Libraries and packages for text (pre-)processing 
import string
import re
import nltk

In [12]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''

reviews = pd.read_csv("IMDB Dataset.csv").iloc[:100,:]


reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
95,Daniel Day-Lewis is the most versatile actor a...,positive
96,My guess would be this was originally going to...,negative
97,"Well, I like to watch bad horror B-Movies, cau...",negative
98,"This IS the worst movie I have ever seen, as w...",negative


In [13]:
#Split the data into train and split

from sklearn.model_selection import train_test_split

In [14]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0
X = reviews.drop(["sentiment"],axis=1)
y = reviews["sentiment"]
y.replace({"positive":1,"negative":0},inplace=True)
train_reviews, test_reviews, train_sentiment, test_sentiment = train_test_split(X,y,random_state=0)

test_sentiment
# train_sentiment.head()

26    1
86    0
2     1
55    0
75    1
93    1
16    1
73    1
54    0
95    1
53    1
92    1
78    0
13    0
7     0
30    1
22    1
24    0
33    1
8     0
43    0
62    1
3     0
71    0
45    1
Name: sentiment, dtype: int64

In [15]:
train_reviews.head()

,review
48,Preston Sturgis' THE POWER AND THE GLORY was u...
6,I sure would like to see a resurrection of a u...
99,I have been a Mario fan for as long as I can r...
82,"Oh God, I must have seen this when I was only ..."
76,The Last Hard Men finds James Coburn an outlaw...


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [16]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    #complete this function that lowers and tokenizes the reviews
    tokenise = list()
    for lis in text_seqs:
        token = encoder(lis)
        tokenise.append([(tok.text).lower() for tok in token])
    return tokenise
    


train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['review'])
train_reviews[['review','Tokenized_Review']][:10]


C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

,review,Tokenized_Review
48,Preston Sturgis' THE POWER AND THE GLORY was u...,"[preston, sturgis, ', the, power, and, the, gl..."
6,I sure would like to see a resurrection of a u...,"[i, sure, would, like, to, see, a, resurrectio..."
99,I have been a Mario fan for as long as I can r...,"[i, have, been, a, mario, fan, for, as, long, ..."
82,"Oh God, I must have seen this when I was only ...","[oh, god, ,, i, must, have, seen, this, when, ..."
76,The Last Hard Men finds James Coburn an outlaw...,"[the, last, hard, men, finds, james, coburn, a..."
60,What happened? What we have here is basically ...,"[what, happened, ?, what, we, have, here, is, ..."
80,Stephen Hawkings is a genius. He is the king o...,"[stephen, hawkings, is, a, genius, ., he, is, ..."
90,"""Down Periscope"" has been in our library since...","["", down, periscope, "", has, been, in, our, li..."
68,I'm not sure why the producers needed to trade...,"[i, 'm, not, sure, why, the, producers, needed..."
51,"***SPOILERS*** All too, in real life as well a...","[*, *, *, spoilers, *, *, *, all, too, ,, in, ..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [17]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle


def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for doc in token_seqs:
        for word in doc:
            try:
                token_counts[word]+=1
            except KeyError:
                token_counts[word]=1


    # Then, assign each word to a numerical index, i.e save  all these words in a list. 
    # Filter words that occur less than or equal to min_freq times.
    
    lexicon = [key for key,value in token_counts.items() if value>min_freq]


#     #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  
#     #index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {lexicon[i]:i+2 for i in range(len(lexicon))}
    
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times

    lexicon_size = len(lexicon)


    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)


LEXICON SAMPLE (1589 total items):
{'sturgis': 2, "'": 3, 'the': 4, 'power': 5, 'and': 6, 'glory': 7, 'was': 8, 'by': 9, 'public': 10, 'for': 11, 'nearly': 12, 'twenty': 13, 'or': 14, 'years': 15, 'until': 16, 'late': 17, '1990s': 18, 'when': 19, 'it': 20, 'even': 21}


###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [18]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in token_seq] for token_seq in token_seqs]
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Tokenized_Review,Review_Idxs
48,"[preston, sturgis, ', the, power, and, the, gl...","[1, 2, 3, 4, 5, 6, 4, 7, 8, 1, 9, 4, 10, 11, 1..."
6,"[i, sure, would, like, to, see, a, resurrectio...","[175, 252, 253, 91, 85, 254, 92, 1, 47, 92, 23..."
99,"[i, have, been, a, mario, fan, for, as, long, ...","[175, 163, 307, 92, 308, 309, 11, 43, 310, 43,..."
82,"[oh, god, ,, i, must, have, seen, this, when, ...","[282, 416, 51, 175, 304, 163, 417, 48, 19, 175..."
76,"[the, last, hard, men, finds, james, coburn, a...","[4, 409, 391, 470, 336, 471, 472, 249, 1, 473,..."
60,"[what, happened, ?, what, we, have, here, is, ...","[289, 533, 534, 289, 121, 163, 42, 44, 535, 92..."
80,"[stephen, hawkings, is, a, genius, ., he, is, ...","[569, 570, 44, 92, 1, 26, 125, 44, 4, 571, 47,..."
90,"["", down, periscope, "", has, been, in, our, li...","[599, 144, 1, 599, 70, 307, 27, 600, 1, 564, 2..."
68,"[i, 'm, not, sure, why, the, producers, needed...","[175, 520, 89, 252, 619, 4, 620, 621, 85, 622,..."
51,"[*, *, *, spoilers, *, *, *, all, too, ,, in, ...","[647, 647, 647, 1, 647, 647, 647, 173, 205, 51..."


In [19]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow_seqs = np.array([np.bincount(arr,minlength=matrix_length) for arr in idx_seqs])

    return bow_seqs
    


bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [[  0 112   3 ...   0   0   0]
 [  0  12   0 ...   0   0   0]
 [  0  57   0 ...   0   0   0]
 ...
 [  0  20   0 ...   0   0   0]
 [  0  28   0 ...   2   2   0]
 [  0  23   0 ...   0   0   2]]
SHAPE: (75, 1590) 



,Word,Count
0,,0
1,<UNK>,112
2,sturgis,3
3,',3
4,the,50
...,...,...
1585,predictably,0
1586,abraham,0
1587,noah,0
1588,canaanites,0


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    
    #find the biggest review's length and save it in the variable below
    max_seq_len = max([len(x) for x in idx_seqs])
    
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    return np.array(pad_sequences(idx_seqs,maxlen=max_seq_len,padding='pre'))


train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[   1    2    3 ...    1  195   26]
 [   0    0    0 ...  171   20   26]
 [   0    0    0 ...   47  297  415]
 ...
 [   0    0    0 ...   92 1064   26]
 [   0    0    0 ...   48  164  415]
 [   0    0    0 ...  450    1  615]]
SHAPE: (75, 804) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [21]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    model = tf.keras.Sequential()
    input_layer = Input(shape=(None,))
    model.add(input_layer)
    # Layer 2
    model.add(Embedding(input_dim=n_input_nodes,output_dim=n_embedding_nodes,mask_zero=True))
    # Layer 3
    model.add(GRU(n_hidden_nodes,return_sequences=True))
    # Layer 4
    model.add(Dense(1,activation='sigmoid'))
    #Specify which layers are input and output, compile model with loss and optimization functions
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model


In [22]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


###  <font color='#6629b2'>Training</font>

In [23]:
train_padded_idxs

array([[   1,    2,    3, ...,    1,  195,   26],
       [   0,    0,    0, ...,  171,   20,   26],
       [   0,    0,    0, ...,   47,  297,  415],
       ...,
       [   0,    0,    0, ...,   92, 1064,   26],
       [   0,    0,    0, ...,   48,  164,  415],
       [   0,    0,    0, ...,  450,    1,  615]])

In [24]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
y_tensor=tf.convert_to_tensor(train_sentiment.values,dtype=tf.int64)

rnn_model.fit(x=train_padded_idxs,y=y_tensor,batch_size=20,epochs=5,steps_per_epoch=50)

Epoch 1/5


KeyboardInterrupt: 

###  <font color='#6629b2'>Prediction</font>

In [ ]:
'''Put test reviews in padded matrix just how we did for train_reviews'''


print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

In [ ]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

###  <font color='#6629b2'>Evaluation</font>

In [ ]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 


On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [ ]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.